<a href="https://colab.research.google.com/github/MuhammadIrzam447/visionCodes/blob/master/mm_imdb(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %cd /content/Dataset(s)/mm-imdb/fused

In [ ]:
!gdown https://drive.google.com/uc?id=1dxd2pySfCIDJYG7qMtuJre8ph068xc1X

In [ ]:
!gdown https://drive.google.com/uc?id=1sYR9EgHkM0oiGRQVlFQCyHO8kMRJ4ibQ

In [ ]:
!gdown https://drive.google.com/uc?id=1X4cmMYRjxXFomCJ1adMhPNMYtd4WeDHP

In [ ]:
!gdown https://drive.google.com/uc?id=1yAeNEPXD3LNxmtSnC09jEg0vEDc1qiaq

In [ ]:
!unzip test.zip

In [ ]:
!unzip train.zip

# Training Dataset Preparation

In [ ]:
import os
image_file_paths = []
genre_labels = []

image_folder_add = ""
labels_file = ''

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        image_file_paths.append(image_path)
        genre_labels.append(labels)

In [ ]:
image_file_paths

In [ ]:
genre_labels

In [ ]:
unique_labels_set = set()

for labels in genre_labels:
    unique_labels_set.update(labels)

unique_labels = sorted(list(unique_labels_set))

In [ ]:
unique_labels

In [ ]:
len(unique_labels)

In [ ]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in genre_labels:
    for label in labels:
        label_counts[label] += 1

In [ ]:
label_counts

In [ ]:
multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in unique_labels]
    multi_hot_labels.append(multi_hot)

In [ ]:
multi_hot_labels[1]

In [ ]:
genre_labels[1]

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomMultiLabelDataset(Dataset):
    def __init__(self, image_file_paths, multi_encoded_labels, transform=None):
        self.image_file_paths = image_file_paths
        self.multi_encoded_labels = multi_encoded_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_file_paths)

    def __getitem__(self, idx):
        image_path = self.image_file_paths[idx]
        image = Image.open(image_path)

        labels = self.multi_encoded_labels[idx]
        labels = torch.tensor(labels, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

In [ ]:
# Create a custom dataset
train_dataset = CustomMultiLabelDataset(image_file_paths, multi_hot_labels)

In [ ]:
len(train_dataset)

In [ ]:
train_dataset[0]

In [ ]:
print(type(train_dataset))

In [ ]:
model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)

In [ ]:
def transform(examples):
    # inputs = image_processor([img.convert("RGB") for img in examples], return_tensors="pt")
    inputs = image_processor(examples, return_tensors="pt")
    inputs["labels"] = examples["label"]
    return inputs

In [ ]:
dataset = [transform(item) for item in train_dataset]

In [ ]:
print(type(train_dataset))

# New Section

Test Dataset

In [ ]:
import os
image_file_paths = []
genre_labels = []

image_folder_add = "/content/test"
labels_file = '/content/test_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        image_file_paths.append(image_path)
        genre_labels.append(labels)

In [ ]:
# image_file_paths

In [ ]:
# unique_labels_set = set()

# for labels in genre_labels:
#     unique_labels_set.update(labels)

# unique_labels = sorted(list(unique_labels_set))

In [ ]:
len(unique_labels)

In [ ]:
multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in unique_labels]
    multi_hot_labels.append(multi_hot)

Training Dataset

In [ ]:
import os
train_image_file_paths = []
train_genre_labels = []

train_image_folder_add = "/content/test"
train_labels_file = '/content/test_label.txt'

with open(train_labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(train_image_folder_add, filename)
        train_image_file_paths.append(image_path)
        train_genre_labels.append(labels)

In [ ]:
unique_labels_set = set()

for labels in train_genre_labels:
    unique_labels_set.update(labels)

unique_labels = sorted(list(unique_labels_set))

In [ ]:
num_labels = len(unique_labels)
label2id = {label: str(i) for i, label in enumerate(unique_labels)}
id2label = {str(i): label for i, label in enumerate(unique_labels)}

In [ ]:
id2label

In [ ]:
num_labels

In [ ]:
train_multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in unique_labels]
    train_multi_hot_labels.append(multi_hot)

Creating Datasets

In [ ]:
!pip install datasets
from datasets import Dataset

In [ ]:
train_data = {'image': train_image_file_paths, 'label': train_multi_hot_labels}
ds_train = Dataset.from_dict(train_data)

val_data = {'image': image_file_paths, 'label': multi_hot_labels}
ds_val = Dataset.from_dict(val_data)


In [ ]:
ds_train

In [ ]:
ds_val

In [ ]:
ds_train['image'][0], ds_val['image'][0]

In [ ]:
# labels = ds.features["label"]
labels = unique_labels
labels

In [ ]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name).to(device)

In [ ]:
import PIL.Image as pil

def transform(examples):
  # inputs = image_processor([Image.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [ ]:
train_dataset = ds_train.with_transform(transform)
val_dataset = ds_val.with_transform(transform)

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
sample = train_dataset[2000]
# img = sample["pixel_values"]
# filename = os.path.basename(img.filename)
# print(f"File Name: {filename}")

In [ ]:
sample

In [ ]:
for item in train_dataset:
  print(item['pixel_values'].shape)
  print(item["labels"])
  break

In [ ]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

Model Creation

In [ ]:
# load the ViT model
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels= len(unique_labels),
    label2id = {label: str(i) for i, label in enumerate(unique_labels)},
    id2label = {str(i): label for i, label in enumerate(unique_labels)},
    problem_type = "multi_label_classification",
    ignore_mismatched_sizes=True,
)

In [ ]:
# Training loop
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

batch_size = 32

In [ ]:
train_dataset_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
valid_dataset_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
log_dir = "/content/Untitled Folder"
summary_writer = SummaryWriter(log_dir=log_dir)

In [ ]:
num_epochs = 20
model = model.to(device)

In [ ]:
n_train_steps = num_epochs * len(train_dataset_loader)
n_valid_steps = len(valid_dataset_loader)
current_step = 0

In [ ]:
n_train_steps , n_valid_steps

In [ ]:
# logging, eval & save steps
save_steps = 1000

In [ ]:
def compute_metrics(eval_pred):
  accuracy_score = accuracy.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids)
  f1_score = f1.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids, average="macro")
  return {**accuracy_score, **f1_score}

In [ ]:
for epoch in range(num_epochs):

    model.train()
    train_loss = 0
    progress_bar = tqdm(range(current_step, n_train_steps), "Training", dynamic_ncols=True, ncols=80)

    for batch in train_dataset_loader:
      if (current_step+1) % save_steps == 0:
        print()
        print(f"Validation at step {current_step}...")
        print()

        model.eval()

        predictions, labels = [], []
        valid_loss = 0

        for batch in valid_dataset_loader:
            pixel_values = batch["pixel_values"].to(device)
            label_ids = batch["labels"].to(device).float()

            outputs = model(pixel_values=pixel_values, labels=label_ids)

            loss = outputs.loss
            valid_loss += loss.item()

            logits = outputs.logits.detach().cpu()

            # predictions.extend(logits.argmax(dim=-1).tolist())
            # labels.extend(label_ids.tolist())
            predictions.extend((outputs > 0.5).int().cpu().numpy())
            labels.extend(labels.int().cpu().numpy())

        # eval_prediction = EvalPrediction(predictions=predictions, label_ids=labels)
        # metrics = compute_metrics(eval_prediction)
        print()
        print(f"Epoch: {epoch}, Step: {current_step}, Train Loss: {train_loss / save_steps:.4f}, " +
              f"Valid Loss: {valid_loss / n_valid_steps:.4f}") # , Accuracy: {metrics['accuracy']}, " + f"F1 Score: {metrics['f1']}")
        print()

        # summary_writer.add_scalar("valid_loss", valid_loss / n_valid_steps, global_step=current_step)
        # summary_writer.add_scalar("accuracy", metrics["accuracy"], global_step=current_step)
        # summary_writer.add_scalar("f1", metrics["f1"], global_step=current_step)

        model.save_pretrained(f"./vit-base-food/checkpoint-{current_step}")
        image_processor.save_pretrained(f"./vit-base-food/checkpoint-{current_step}")

        accuracy = accuracy_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions, average='macro')
        recall = recall_score(true_labels, predictions, average='macro')
        f1 = f1_score(true_labels, predictions, average='macro')

        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-Score: {f1}")
        print(classification_report(true_labels, predictions))

        model.train()
        train_loss, valid_loss = 0, 0

      pixel_values = batch["pixel_values"].to(device)
      labels = batch["labels"].to(device).float()

      outputs = model(pixel_values=pixel_values, labels=labels)

      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      loss_v = loss.item()
      train_loss += loss_v

      current_step += 1
      progress_bar.update(1)
      summary_writer.add_scalar("train_loss", loss_v, global_step=current_step)

In [ ]:
for batch in valid_dataset_loader:
            # get the batch
            pixel_values = batch["pixel_values"].to(device)
            label_ids = batch["labels"].to(device).float()
            print(label_ids)
            break